In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-12 13:57:35--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-12 13:57:36 (52.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip


--2023-06-12 13:57:28--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.128, 74.125.24.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   722KB/s    in 0.8s    

2023-06-12 13:57:29 (722 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
unzip_data("nlp_getting_started.zip")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df_shuffled = train_df.sample(frac = 1, random_state=42)

In [ ]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(), train_df_shuffled["target"].to_numpy(), test_size=0.1, random_state = 42)

In [ ]:
type(train_sentences)

numpy.ndarray

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vector = TextVectorization(max_tokens=1000, standardize = "lower_and_strip_punctuation", split = "whitespace", ngrams = None, output_mode = "int", output_sequence_length=15 , pad_to_max_tokens=True)


In [ ]:
text_vector.adapt(train_sentences)

In [ ]:
sample = "There's a flood in my street"
text_vector([sample])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = 10000, output_dim = 128, embeddings_initializer = "uniform", input_length = 15)

In [ ]:
embedding(text_vector([sample]))

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01143087,  0.02805254, -0.00316989, ...,  0.01834658,
         -0.04234991,  0.04274556],
        [ 0.00972005,  0.03012475,  0.02519501, ...,  0.0495686 ,
          0.00382805,  0.03685958],
        [-0.04279635,  0.00759071, -0.04137643, ...,  0.04395026,
          0.01209272,  0.03423469],
        ...,
        [ 0.02870793, -0.04246329, -0.00066205, ...,  0.04382709,
          0.02391306, -0.00531869],
        [ 0.02870793, -0.04246329, -0.00066205, ...,  0.04382709,
          0.02391306, -0.00531869],
        [ 0.02870793, -0.04246329, -0.00066205, ...,  0.04382709,
          0.02391306, -0.00531869]]], dtype=float32)>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [ ]:
model = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB()),

])

model.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
score = model.score(val_sentences, val_labels)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
def calculate_result(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)*100
  model_precision, model_recall, model_f1score, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")
  model_results = {"accuracy":model_accuracy, "precision": model_precision, "recall": model_recall, "model_f1score":model_f1score}

  return model_results




In [ ]:
preds = model.predict(val_sentences)

In [ ]:
result = calculate_result(y_true = val_labels, y_pred = preds)

In [ ]:
result

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'model_f1score': 0.7862189758049549}

In [ ]:
from helper_functions import create_tensorboard_callback

save_dir = "model_logs"


In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vector(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs)


In [ ]:
model_1.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr = 0.001), metrics = ["accuracy"])


In [ ]:
model_1_history = model_1.fit(x = train_sentences, y = train_labels, epochs = 5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(dir_name = save_dir, experiment_name =   "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230612-181448
Epoch 1/5
215/215 [==============================] - 30s 105ms/step - loss: 0.6213 - accuracy: 0.6688 - val_loss: 0.5590 - val_accuracy: 0.7336
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4923 - accuracy: 0.7834 - val_loss: 0.5046 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.4372 - accuracy: 0.8086 - val_loss: 0.4894 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4102 - accuracy: 0.8209 - val_loss: 0.4905 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3945 - accuracy: 0.8253 - val_loss: 0.4975 - val_accuracy: 0.7808


In [ ]:
model_1_pred_probs = tf.squeeze(tf.round(model_1.predict(val_sentences)))

24/24 [==============================] - 0s 2ms/step


In [ ]:
model_1_pred_probs


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [ ]:
result = calculate_result(y_true = val_labels, y_pred = model_1_pred_probs)

In [ ]:
result

{'accuracy': 78.08398950131233,
 'precision': 0.7909718300891369,
 'recall': 0.7808398950131233,
 'model_f1score': 0.7759864846948408}

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vector(inputs)
x = embedding(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs)

In [ ]:
model_2.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr = 0.001), metrics = ["accuracy"])


In [ ]:
model_2.fit(train_sentences, train_labels, epochs = 5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(save_dir, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230612-181528
Epoch 1/5
215/215 [==============================] - 15s 54ms/step - loss: 0.4078 - accuracy: 0.8228 - val_loss: 0.4970 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3763 - accuracy: 0.8340 - val_loss: 0.5162 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3701 - accuracy: 0.8361 - val_loss: 0.5002 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3605 - accuracy: 0.8444 - val_loss: 0.5264 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 2s 12ms/step - loss: 0.3358 - accuracy: 0.8545 - val_loss: 0.5480 - val_accuracy: 0.7677


In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.56987935],
       [0.76388544],
       [0.9140713 ],
       [0.03855203],
       [0.11894756],
       [0.996385  ],
       [0.8963069 ],
       [0.8334121 ],
       [0.7852568 ],
       [0.10307001]], dtype=float32)

In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [ ]:
results = calculate_result(val_labels, model_2_preds)
results

{'accuracy': 76.77165354330708,
 'precision': 0.7691343474319641,
 'recall': 0.7677165354330708,
 'model_f1score': 0.7656749923220023}

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vector(inputs)
x = embedding(x)
x =  layers.GRU(64)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs)


In [ ]:
model_3.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr = 0.003), metrics = ["accuracy"])

In [ ]:
model_3_history = model_3.fit(train_sentences, train_labels, epochs = 5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(save_dir, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230612-181555
Epoch 1/5
215/215 [==============================] - 15s 58ms/step - loss: 0.3918 - accuracy: 0.8237 - val_loss: 0.5153 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3435 - accuracy: 0.8536 - val_loss: 0.5153 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3242 - accuracy: 0.8651 - val_loss: 0.5127 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3031 - accuracy: 0.8705 - val_loss: 0.5841 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2879 - accuracy: 0.8854 - val_loss: 0.5723 - val_accuracy: 0.7625


In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs

24/24 [==============================] - 0s 3ms/step


array([[7.94401705e-01],
       [4.27934319e-01],
       [9.73698676e-01],
       [6.99499622e-02],
       [8.20251554e-02],
       [9.97253358e-01],
       [9.61467385e-01],
       [9.32822704e-01],
       [8.51931155e-01],
       [1.47150770e-01],
       [2.02115491e-01],
       [3.43560994e-01],
       [3.64079177e-02],
       [1.60022244e-01],
       [7.99600501e-03],
       [1.11840613e-01],
       [1.41615244e-02],
       [3.50720614e-01],
       [9.98531505e-02],
       [3.86040062e-01],
       [9.64664519e-01],
       [2.24527661e-02],
       [2.99862176e-01],
       [4.81365286e-02],
       [8.43616962e-01],
       [9.98610377e-01],
       [4.62958589e-02],
       [2.08582357e-02],
       [1.45779746e-02],
       [2.00992674e-02],
       [9.29745138e-01],
       [2.81280637e-01],
       [1.64188266e-01],
       [7.98454583e-01],
       [5.88842183e-02],
       [8.36466625e-02],
       [9.98490214e-01],
       [9.01881605e-02],
       [4.66523655e-02],
       [9.87582207e-01],


In [ ]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))

In [ ]:
results = calculate_result(val_labels, model_3_preds)
results

{'accuracy': 76.24671916010499,
 'precision': 0.7639905060800456,
 'recall': 0.7624671916010499,
 'model_f1score': 0.7602456873168202}

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vector(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs)

In [ ]:
model_4.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr = 0.003), metrics =  ["accuracy"])

In [ ]:
mdoel_4_history = model_4.fit(train_sentences, train_labels, epochs = 5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(save_dir, "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230612-181621
Epoch 1/5
215/215 [==============================] - 15s 50ms/step - loss: 0.3560 - accuracy: 0.8488 - val_loss: 0.5658 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.2923 - accuracy: 0.8802 - val_loss: 0.6148 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2693 - accuracy: 0.8866 - val_loss: 0.5918 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2472 - accuracy: 0.9012 - val_loss: 0.6855 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2341 - accuracy: 0.9021 - val_loss: 0.7793 - val_accuracy: 0.7467


In [ ]:
model_4_pred_probs = model_4.predict(val_sentences)

24/24 [==============================] - 1s 3ms/step


In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [ ]:
result = calculate_result(val_labels, model_4_preds)
results

{'accuracy': 76.24671916010499,
 'precision': 0.7639905060800456,
 'recall': 0.7624671916010499,
 'model_f1score': 0.7602456873168202}

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vector(inputs)
x = embedding(x)
x = layers.Conv1D(64, 5, activation = "relu", strides = 1,padding = "valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs)

model_5.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr = 0.0003), metrics = ["accuracy"])
model_5.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_5 (Dense)             (None, 1)                 65  

In [ ]:
model_5_history = model_5.fit(train_sentences, train_labels, epochs=5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(save_dir, "model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20230612-181648
Epoch 1/5
215/215 [==============================] - 15s 46ms/step - loss: 0.3703 - accuracy: 0.8374 - val_loss: 0.5472 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3352 - accuracy: 0.8584 - val_loss: 0.5825 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3081 - accuracy: 0.8723 - val_loss: 0.5612 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2782 - accuracy: 0.8879 - val_loss: 0.6202 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2485 - accuracy: 0.9007 - val_loss: 0.6504 - val_accuracy: 0.7559


In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs

24/24 [==============================] - 0s 3ms/step


array([[8.08872938e-01],
       [8.33545387e-01],
       [8.11166525e-01],
       [4.30208184e-02],
       [2.74651021e-01],
       [9.99960899e-01],
       [2.18663931e-01],
       [9.88709509e-01],
       [2.84027934e-01],
       [1.25963673e-01],
       [6.61273822e-02],
       [4.45665866e-01],
       [3.42692472e-02],
       [2.30247170e-01],
       [1.10264020e-02],
       [3.89830619e-02],
       [2.60299258e-02],
       [1.74599171e-01],
       [2.36104906e-01],
       [6.08399332e-01],
       [9.94816482e-01],
       [1.37186318e-04],
       [2.83807695e-01],
       [6.55152183e-03],
       [8.83741140e-01],
       [9.96313989e-01],
       [5.91544434e-03],
       [1.43702782e-03],
       [1.40686529e-02],
       [1.01389773e-01],
       [9.91747499e-01],
       [9.94260386e-02],
       [7.93132000e-03],
       [4.02463585e-01],
       [7.27238953e-02],
       [1.10156037e-01],
       [9.99749362e-01],
       [3.77759188e-02],
       [3.19909602e-02],
       [4.03692752e-01],


In [ ]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

In [ ]:
model_5_result = calculate_result(val_labels, model_5_preds)
model_5_result

{'accuracy': 75.59055118110236,
 'precision': 0.7573698742202679,
 'recall': 0.7559055118110236,
 'model_f1score': 0.7535524954357875}

In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample, "This converts sentences into numbers"])

In [ ]:
sentences_encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape = [], dtype = tf.string, trainable = False)
model_6 = tf.keras.Sequential([
    sentences_encoder,
    layers.Dense(1, activation = "sigmoid")
])
model_6.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr = 0.0003), metrics = ["accuracy"])


In [ ]:
model_6_history = model_6.fit(train_sentences, train_labels, epochs = 5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(save_dir, "tf_hub_sentences_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentences_encoder/20230612-181825
Epoch 1/5
215/215 [==============================] - 7s 19ms/step - loss: 0.6480 - accuracy: 0.7343 - val_loss: 0.6128 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.5810 - accuracy: 0.7870 - val_loss: 0.5633 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.5381 - accuracy: 0.7929 - val_loss: 0.5313 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.5095 - accuracy: 0.7968 - val_loss: 0.5104 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4895 - accuracy: 0.8000 - val_loss: 0.4961 - val_accuracy: 0.7887


In [ ]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred = tf.squeeze(tf.round(model_6_pred_probs))
model_6_pred

24/24 [==============================] - 1s 10ms/step


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [ ]:
model_6_result = calculate_result(val_labels, model_6_pred)
model_6_result

{'accuracy': 78.87139107611549,
 'precision': 0.7893127453028103,
 'recall': 0.7887139107611548,
 'model_f1score': 0.787504696224858}

In [ ]:
train_10_percent_split = int(0.1 *len(train_sentences))
train_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]


In [ ]:
model_7 = tf.keras.models.clone_model(model_6)

In [ ]:
model_7.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr = 0.0003), metrics = ["accuracy"])

In [ ]:
model_7_history = model_7.fit(train_10_percent, train_labels_10_percent, epochs= 5, validation_data = (val_sentences, val_labels), callbacks=[create_tensorboard_callback(save_dir, "model_7_10_percent")])

Saving TensorBoard log files to: model_logs/model_7_10_percent/20230612-181853
Epoch 1/5
22/22 [==============================] - 4s 47ms/step - loss: 0.6863 - accuracy: 0.5985 - val_loss: 0.6836 - val_accuracy: 0.6168
Epoch 2/5
22/22 [==============================] - 1s 25ms/step - loss: 0.6736 - accuracy: 0.6759 - val_loss: 0.6745 - val_accuracy: 0.6654
Epoch 3/5
22/22 [==============================] - 1s 27ms/step - loss: 0.6619 - accuracy: 0.7255 - val_loss: 0.6660 - val_accuracy: 0.6955
Epoch 4/5
22/22 [==============================] - 1s 27ms/step - loss: 0.6508 - accuracy: 0.7562 - val_loss: 0.6578 - val_accuracy: 0.7178
Epoch 5/5
22/22 [==============================] - 1s 24ms/step - loss: 0.6405 - accuracy: 0.7708 - val_loss: 0.6497 - val_accuracy: 0.7257


In [ ]:
model_7_pred_probs = model_7.predict(val_sentences)

24/24 [==============================] - 1s 10ms/step


In [ ]:
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))

In [ ]:
model_7_result = calculate_result(val_labels, model_7_preds)
model_7_result

{'accuracy': 72.57217847769029,
 'precision': 0.7525701752909861,
 'recall': 0.7257217847769029,
 'model_f1score': 0.7112114207853204}

In [ ]:
!tensorboard dev upload --logdir ./model_logs/ \
  --name "NLP Modelling Experiment" \
  --description "comapring multiple architecture" \
  --one_shot

2023-06-12 18:19:04.261085: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./model_logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) 

In [ ]:
model_6.save("model_6.h5")


In [ ]:
import tensorflow_hub as hub
loaded_model_6 = tf.keras.models.load_model(
       ("model_6.h5"),
       custom_objects={'KerasLayer':hub.KerasLayer}
)

In [ ]:
loaded_model_6.evaluate(val_sentences, val_labels)

24/24 [==============================] - 1s 23ms/step - loss: 0.4961 - accuracy: 0.7887


[0.4961186349391937, 0.7887139320373535]

In [ ]:
model_6.save("model_6_save_format")

In [ ]:
load_model_6_saved_format = tf.keras.models.load_model("model_6_save_format")

In [ ]:
load_model_6_saved_format.evaluate(val_sentences, val_labels)

24/24 [==============================] - 1s 12ms/step - loss: 0.4961 - accuracy: 0.7887


[0.4961186349391937, 0.7887139320373535]

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
!unzip 08_model_6_USE_feature_extractor.zip

--2023-06-11 18:23:56--  https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.119.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 960779165 (916M) [application/zip]
Saving to: ‘08_model_6_USE_feature_extractor.zip’

08_model_6_USE_feat 100%[===================>] 916.27M  32.8MB/s    in 29s     

2023-06-11 18:24:24 (32.1 MB/s) - ‘08_model_6_USE_feature_extractor.zip’ saved [960779165/960779165]

Archive:  08_model_6_USE_feature_extractor.zip
   creating: 08_model_6_USE_feature_extractor/
   creating: 08_model_6_USE_feature_extractor/assets/
   creating: 08_model_6_USE_feature_extractor/variables/
  inflating: 08_model_6_USE_feature_extractor/variables/variables.data-00000-of-00001  
  inflating: 08_model_6_USE_feature_extractor/variables/variables

In [ ]:
model_6_pretrained = tf.keras.models.load_model("08_model_6_USE_feature_extractor")

In [ ]:
model_6_pretrained.evaluate(val_sentences, val_labels)

24/24 [==============================] - 2s 22ms/step - loss: 0.4272 - accuracy: 0.8163


[0.42723119258880615, 0.8162729740142822]

In [ ]:
model_6_pretrained_preds_probs = model_6_pretrained.predict(val_sentences)
model_6_pretrained_preds = tf.squeeze(tf.round(model_6_pretrained_preds_probs))

24/24 [==============================] - 1s 16ms/step


In [ ]:
val_df = pd.DataFrame({"text": val_sentences, "target": val_labels, "pred": model_6_pretrained_preds, "pred_probs": tf.squeeze(model_6_pretrained_preds_probs)})

In [ ]:
val_df

,text,target,pred,pred_probs
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.159757
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.747162
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.988749
3,@camilacabello97 Internally and externally scr...,1,0.0,0.196229
4,Radiation emergency #preparedness starts with ...,1,1.0,0.707808
...,...,...,...,...
757,That's the ultimate road to destruction,0,0.0,0.137074
758,@SetZorah dad why dont you claim me that mean ...,0,0.0,0.095865
759,FedEx will no longer transport bioterror patho...,0,1.0,0.876982
760,Crack in the path where I wiped out this morni...,0,1.0,0.612631


In [ ]:
import random
test_sentences = test_df["text"].to_list()
test_samples = random.sample(test_sentences, 10)
for test_sample in test_samples:
  pred_prob = tf.squeeze(model_6_pretrained.predict([test_sample]))
  pred = tf.round(pred_prob)
  print(int(pred))
  print()

1/1 [==============================] - 0s 154ms/step
1

1/1 [==============================] - 0s 80ms/step
0

1/1 [==============================] - 0s 58ms/step
0

1/1 [==============================] - 0s 60ms/step
0

1/1 [==============================] - 0s 57ms/step
1

1/1 [==============================] - 0s 55ms/step
1

1/1 [==============================] - 0s 55ms/step
0

1/1 [==============================] - 0s 58ms/step
1

1/1 [==============================] - 0s 46ms/step
0

1/1 [==============================] - 0s 36ms/step
1

